In [1]:
!pip install duckdb
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.6-cp311-cp311-linux_x86_64.whl size=4022120 sha256=74813fb056bd3505eaafddc559fff3f23580cf12eb746ffffd40a20784ca747b
  Stored in directory: /root/.cache/pip/wheels/e8/96/d2/acfb576f7a58ef0580e2fec8096e5eefd17cc356017089337b
Successfully built llama-cpp-python


In [2]:
from huggingface_hub import hf_hub_download
model_name = "motherduckdb/DuckDB-NSQL-7B-v0.1-GGUF"
model_file = "DuckDB-NSQL-7B-v0.1-q8_0.gguf"
model_path = hf_hub_download(model_name,
                             filename=model_file,
                             local_dir='/content')
model_path

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DuckDB-NSQL-7B-v0.1-q8_0.gguf:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

'/content/DuckDB-NSQL-7B-v0.1-q8_0.gguf'

In [3]:
from llama_cpp import Llama

llm = Llama(model_path=model_path, n_gpu_layers=-1)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/DuckDB-NSQL-7B-v0.1-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count 

In [4]:
import requests
import pandas as pd
import json
df_csv = pd.read_csv('/content/user_data.csv')
df_csv.head()

,user_id,first_name,last_name,email,city,country,age,annual_income,job_title,department
0,1,John,Doe,john.doe@example.com,New York,USA,34,85000,Software Engineer,IT
1,2,Jane,Smith,jane.smith@example.com,San Francisco,USA,29,97000,Data Scientist,Analytics
2,3,Mike,Brown,mike.brown@example.com,London,UK,45,62000,Project Manager,Operations
3,4,Emily,Clark,emily.clark@example.com,Sydney,Australia,31,78000,Marketing Specialist,Marketing
4,5,Chris,Johnson,chris.johnson@example.com,Toronto,Canada,38,91000,Sales Manager,Sales


In [6]:
import pandas as pd
import duckdb

# Example DataFrame
data = {'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']}
df = pd.DataFrame(data)

# Connect to DuckDB
con = duckdb.connect()

# Register the DataFrame as a DuckDB table
# con.register('temp_df', df)

# Create a DuckDB table from the DataFrame
table_name = 'temp_df'
con.execute(f"DROP TABLE IF EXISTS {table_name}")  # Optional: drop the table if it exists
con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df_csv")

# # Verify that the table has been created
# print(con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf())


# Create a table from the registered DataFrame
con.execute("CREATE TABLE data AS SELECT * FROM temp_df LIMIT 1;")


# Fetch the DDL (CREATE TABLE statement) for the table 'data'
result = con.execute("SELECT sql FROM duckdb_tables() WHERE table_name = 'data';").df()
ddl_create = result.iloc[0, 0]

# Close the connection
# con.close()

# Print the DDL statement
print(ddl_create)

tables = con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchdf()

query = "Can I get the list of people working in the Design Department?"


prompt = """### Instruction:
   Your task is to generate valid duckdb SQL to answer the following question.
   ### Input:
   Here is the database schema that the SQL query will run on:
   {ddl_create}

   ### Question:
   {query_input}
   ### Response (use duckdb shorthand if possible):
   """

prompt = prompt.format(ddl_create=ddl_create, query_input=query)
# print(prompt)

pred = llm(prompt, temperature=0.1, max_tokens=1000)
sql_output = pred["choices"][0]["text"]

sql_output = sql_output.replace("FROM data", f"FROM user_data")
print(sql_output)


# con = duckdb.connect()
# try:
#    query_result = con.sql(sql_output).df()
# except Exception as error:
#    print(f"Could not execute SQL query {error=}")
# finally:
#    con.close()

# print(query_result)


CREATE TABLE "data"(user_id BIGINT, first_name VARCHAR, last_name VARCHAR, email VARCHAR, city VARCHAR, country VARCHAR, age BIGINT, annual_income BIGINT, job_title VARCHAR, department VARCHAR);


Llama.generate: 113 prefix-match hit, remaining 23 prompt tokens to eval
llama_perf_context_print:        load time =   50833.62 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    23 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    11 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19094.61 ms /    34 tokens


 SELECT * FROM user_data WHERE department = 'Design';
